## TODOs
1. time things
1. put summaries for the algorithms to explain what we did. This can also be taken later from the report

## Inputs

1. restaurant dimensions
1. number of timeslots?
1. marginal revenue function
1. cost of goodwill
1. randomly generated data - boolean
1. maximum table size
1. objective? - enum

## Construction Heuristic

### Algorithm

1. sort parties in order of descending size
1. add parties to the timeslot k until the restaurant capacity is full or no more tables can be added

#### Pre-work: Randomly generating a unique HEX string of 6 digits to use as a file identifier and relate logging files to data files

In [1]:
import random
import string

RG_HEX_ID = ''.join(random.choice(string.hexdigits) for _ in range(6))
print(RG_HEX_ID)

8ACa14


### Step 1: Parse input parameters

In [2]:
import json
import random

w = l = customer_goodwill = REVENUE_GOAL = num_timeslots = randomly_generated_data = 0
marginal_revenue = {}

# set of timeslots - static
K = 7

with open('../data.json') as f:
    data = json.load(f)
    w = data['restaurant_width']
    l = data['restaurant_length']
    customer_goodwill = data['customer_goodwill']
    REVENUE_GOAL = data['revenue_goal']
    randomly_generated_data = data['randomly_generated_data']
    max_party_size = data['max_party_size']
    for i in data['marginal_revenue']:
        marginal_revenue[int(i)] = data['marginal_revenue'][i]
        
# Restaurant squared area
MAX_SIZE = l * w

### Step 2: Randomly generate data

In [3]:
# set of parties at each timeslot
# since there are 7 timeslots and since this data is randomly generated, 
# the set of parties during timelsot k will be I[k].
# Prior to having randomly generated demand we had set I = 40 (40 parties) for each timeslot
if randomly_generated_data == True:
    I = [
        random.randint(8,10), 
        random.randint(10,12),
        random.randint(12,14),
        random.randint(14,16),
        random.randint(14,16),
        random.randint(14,15),
        random.randint(10,12),
    ]
    
else:
    print("Please allow data to be randomly generated to ensure accuracy and reliability of results")


#initializing the party 
party_size = {}
for k in range(K):
    for i in range(I[k]):
        party_size[i, k] = random.randint(2, max_party_size)
        
print(f"{max_party_size}\n")
print(f"{party_size}")

8

{(0, 0): 6, (1, 0): 5, (2, 0): 2, (3, 0): 3, (4, 0): 8, (5, 0): 5, (6, 0): 6, (7, 0): 2, (0, 1): 5, (1, 1): 2, (2, 1): 3, (3, 1): 8, (4, 1): 2, (5, 1): 3, (6, 1): 8, (7, 1): 8, (8, 1): 6, (9, 1): 8, (0, 2): 7, (1, 2): 8, (2, 2): 2, (3, 2): 2, (4, 2): 4, (5, 2): 5, (6, 2): 6, (7, 2): 6, (8, 2): 5, (9, 2): 6, (10, 2): 3, (11, 2): 6, (12, 2): 2, (0, 3): 2, (1, 3): 2, (2, 3): 8, (3, 3): 3, (4, 3): 8, (5, 3): 5, (6, 3): 4, (7, 3): 7, (8, 3): 7, (9, 3): 2, (10, 3): 2, (11, 3): 2, (12, 3): 6, (13, 3): 3, (14, 3): 7, (0, 4): 5, (1, 4): 7, (2, 4): 2, (3, 4): 3, (4, 4): 5, (5, 4): 7, (6, 4): 4, (7, 4): 4, (8, 4): 8, (9, 4): 7, (10, 4): 8, (11, 4): 5, (12, 4): 5, (13, 4): 8, (14, 4): 2, (15, 4): 7, (0, 5): 6, (1, 5): 8, (2, 5): 6, (3, 5): 7, (4, 5): 8, (5, 5): 8, (6, 5): 5, (7, 5): 3, (8, 5): 2, (9, 5): 3, (10, 5): 4, (11, 5): 6, (12, 5): 3, (13, 5): 2, (14, 5): 6, (0, 6): 8, (1, 6): 6, (2, 6): 7, (3, 6): 5, (4, 6): 3, (5, 6): 5, (6, 6): 2, (7, 6): 2, (8, 6): 6, (9, 6): 2}


### Step 3: Calculate table sizes and space that each party will occupy if seated

In [4]:
# the table mapping below represents
# how many tables a specific party_size would require
table_mapping = {}
for i in range(2,max_party_size + 1):
    table_mapping[i] = i // 2 + i % 2

# initializing the amount of space that each of the party sizes 
# would take up
space = {}
for k in range(K):
    for i in range(I[k]):
        num_tables = table_mapping.get(party_size[i, k])
        space[i, k] = 6 + 3 * num_tables

### Step 4: Create and pre-populate data structures

In [5]:
sorted_space = space.copy()

sorted_space = {k: v for k, v in sorted(sorted_space.items(), key=lambda item: item[1], reverse=True)}
sorted_space = {k: v for k, v in sorted(sorted_space.items(), key=lambda item: item[0][1], reverse=False)}
print(f"{sorted_space}")

{(4, 0): 18, (0, 0): 15, (1, 0): 15, (5, 0): 15, (6, 0): 15, (3, 0): 12, (2, 0): 9, (7, 0): 9, (3, 1): 18, (6, 1): 18, (7, 1): 18, (9, 1): 18, (0, 1): 15, (8, 1): 15, (2, 1): 12, (5, 1): 12, (1, 1): 9, (4, 1): 9, (0, 2): 18, (1, 2): 18, (5, 2): 15, (6, 2): 15, (7, 2): 15, (8, 2): 15, (9, 2): 15, (11, 2): 15, (4, 2): 12, (10, 2): 12, (2, 2): 9, (3, 2): 9, (12, 2): 9, (2, 3): 18, (4, 3): 18, (7, 3): 18, (8, 3): 18, (14, 3): 18, (5, 3): 15, (12, 3): 15, (3, 3): 12, (6, 3): 12, (13, 3): 12, (0, 3): 9, (1, 3): 9, (9, 3): 9, (10, 3): 9, (11, 3): 9, (1, 4): 18, (5, 4): 18, (8, 4): 18, (9, 4): 18, (10, 4): 18, (13, 4): 18, (15, 4): 18, (0, 4): 15, (4, 4): 15, (11, 4): 15, (12, 4): 15, (3, 4): 12, (6, 4): 12, (7, 4): 12, (2, 4): 9, (14, 4): 9, (1, 5): 18, (3, 5): 18, (4, 5): 18, (5, 5): 18, (0, 5): 15, (2, 5): 15, (6, 5): 15, (11, 5): 15, (14, 5): 15, (7, 5): 12, (9, 5): 12, (10, 5): 12, (12, 5): 12, (8, 5): 9, (13, 5): 9, (0, 6): 18, (2, 6): 18, (1, 6): 15, (3, 6): 15, (5, 6): 15, (8, 6): 15, 

In [6]:
# initializing a list of keys for the space dictionary
keys_list = list(sorted_space)


# looping over each of the time slot's sorted party sizes 
timeslot_sorted_parties = {}
global_counter = 0
for k in range(K):
    counter = 0
    sorted_parties=[]
    for key in sorted_space.keys():
        sorted_parties.append(sorted_space[keys_list[global_counter]])
        counter += 1
        global_counter += 1
        if(counter == I[k]):
            break
    timeslot_sorted_parties[k] = sorted_parties
print(f"{timeslot_sorted_parties}")
#print(f"{sorted_space}")
#print(f"{sorted_space[sorted_space[i],i]}")

x = [[0 for i in range(I[k])] for k in range(K)]

{0: [18, 15, 15, 15, 15, 12, 9, 9], 1: [18, 18, 18, 18, 15, 15, 12, 12, 9, 9], 2: [18, 18, 15, 15, 15, 15, 15, 15, 12, 12, 9, 9, 9], 3: [18, 18, 18, 18, 18, 15, 15, 12, 12, 12, 9, 9, 9, 9, 9], 4: [18, 18, 18, 18, 18, 18, 18, 15, 15, 15, 15, 12, 12, 12, 9, 9], 5: [18, 18, 18, 18, 15, 15, 15, 15, 15, 12, 12, 12, 12, 9, 9], 6: [18, 18, 15, 15, 15, 15, 12, 9, 9, 9]}


### Step 5: Construction Heuristic

In [7]:
import pprint
import time

start_time = time.time()

# Observation: Sometimes we have a party size of 3/5 instead of 4/6 get allocated even though 4 will produce more revenue.
# This is because the space occupied by each is the same.
# Way we can do this is by sorting at party_size instead of space and that will propogate.
binaries = []
incumbent = []

daily_revenue = 0
counter = 0
space_used = []
for k in range(K):
    state = []
    revenue = 0
    occupied_space = 0
    print(f"timeslot{k}")
    for i in range(I[k]):
        
        dict_to_append = {}
        size_of_party = party_size[keys_list[counter]]
        dict_to_append['party'] = i
        dict_to_append['timeslot'] = k
        dict_to_append['party_size'] = size_of_party
        
        if (occupied_space + timeslot_sorted_parties[k][i] < MAX_SIZE):
            occupied_space += timeslot_sorted_parties[k][i]
            x[k][i] = 1
            added_revenue = marginal_revenue[size_of_party] * size_of_party
            revenue += added_revenue
            #dict_to_append[i,k] = [1, added_revenue]
            binaries.append([i,size_of_party,k,1,added_revenue,revenue])
            dict_to_append['seated'] = 1
            print(f"party size = {size_of_party}, marginal revenue = {marginal_revenue[size_of_party]}, rev={revenue}")
        else:
            dict_to_append['seated'] = 0
            binaries.append([i,size_of_party,k,'',0,revenue])
        counter+=1
        state.append(dict_to_append)
    incumbent.append(state)
    space_used.append(occupied_space)
    print(f"{occupied_space}\n")
incumbent.append(state)
incumbent.append(revenue)
incumbent.append(space_used)
print(f"{incumbent}")

print("\n\n--- %s seconds ---\n\n" % (time.time() - start_time))

timeslot0
party size = 8, marginal revenue = 30, rev=240
party size = 6, marginal revenue = 28, rev=408
party size = 5, marginal revenue = 24, rev=528
party size = 5, marginal revenue = 24, rev=648
party size = 6, marginal revenue = 28, rev=816
party size = 3, marginal revenue = 23, rev=885
party size = 2, marginal revenue = 26, rev=937
99

timeslot1
party size = 8, marginal revenue = 30, rev=240
party size = 8, marginal revenue = 30, rev=480
party size = 8, marginal revenue = 30, rev=720
party size = 8, marginal revenue = 30, rev=960
party size = 5, marginal revenue = 24, rev=1080
party size = 3, marginal revenue = 23, rev=1149
99

timeslot2
party size = 7, marginal revenue = 29, rev=203
party size = 8, marginal revenue = 30, rev=443
party size = 5, marginal revenue = 24, rev=563
party size = 6, marginal revenue = 28, rev=731
party size = 6, marginal revenue = 28, rev=899
party size = 5, marginal revenue = 24, rev=1019
96

timeslot3
party size = 8, marginal revenue = 30, rev=240
party

### Step 6: Export results to CSV

In [8]:
for i in binaries:
    print(f"{i}")

#for key in sorted_space.keys():
    #print(f"{key}")
 #   print(f"{x[key[0]][key[1]]}")

[0, 8, 0, 1, 240, 240]
[1, 6, 0, 1, 168, 408]
[2, 5, 0, 1, 120, 528]
[3, 5, 0, 1, 120, 648]
[4, 6, 0, 1, 168, 816]
[5, 3, 0, 1, 69, 885]
[6, 2, 0, 1, 52, 937]
[7, 2, 0, '', 0, 937]
[0, 8, 1, 1, 240, 240]
[1, 8, 1, 1, 240, 480]
[2, 8, 1, 1, 240, 720]
[3, 8, 1, 1, 240, 960]
[4, 5, 1, 1, 120, 1080]
[5, 6, 1, '', 0, 1080]
[6, 3, 1, 1, 69, 1149]
[7, 3, 1, '', 0, 1149]
[8, 2, 1, '', 0, 1149]
[9, 2, 1, '', 0, 1149]
[0, 7, 2, 1, 203, 203]
[1, 8, 2, 1, 240, 443]
[2, 5, 2, 1, 120, 563]
[3, 6, 2, 1, 168, 731]
[4, 6, 2, 1, 168, 899]
[5, 5, 2, 1, 120, 1019]
[6, 6, 2, '', 0, 1019]
[7, 6, 2, '', 0, 1019]
[8, 4, 2, '', 0, 1019]
[9, 3, 2, '', 0, 1019]
[10, 2, 2, '', 0, 1019]
[11, 2, 2, '', 0, 1019]
[12, 2, 2, '', 0, 1019]
[0, 8, 3, 1, 240, 240]
[1, 8, 3, 1, 240, 480]
[2, 7, 3, 1, 203, 683]
[3, 7, 3, 1, 203, 886]
[4, 7, 3, 1, 203, 1089]
[5, 5, 3, '', 0, 1089]
[6, 6, 3, '', 0, 1089]
[7, 3, 3, '', 0, 1089]
[8, 4, 3, '', 0, 1089]
[9, 3, 3, '', 0, 1089]
[10, 2, 3, 1, 52, 1141]
[11, 2, 3, '', 0, 1141]
[12, 2

In [9]:
import csv
import string

file_name = test = "construction_heuristic_" + RG_HEX_ID + ".csv"
with open(file_name, 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["party i", "size of Party i", "timeslot k", "seated (binary)", "added revenue", "cum. revenue"])
    for i in binaries:
       writer.writerow(i)

#### Visualizing incumbent data structure

In [10]:
pprint.pprint(f"{incumbent}")

("[[{'party': 0, 'timeslot': 0, 'party_size': 8, 'seated': 1}, {'party': 1, "
 "'timeslot': 0, 'party_size': 6, 'seated': 1}, {'party': 2, 'timeslot': 0, "
 "'party_size': 5, 'seated': 1}, {'party': 3, 'timeslot': 0, 'party_size': 5, "
 "'seated': 1}, {'party': 4, 'timeslot': 0, 'party_size': 6, 'seated': 1}, "
 "{'party': 5, 'timeslot': 0, 'party_size': 3, 'seated': 1}, {'party': 6, "
 "'timeslot': 0, 'party_size': 2, 'seated': 1}, {'party': 7, 'timeslot': 0, "
 "'party_size': 2, 'seated': 0}], [{'party': 0, 'timeslot': 1, 'party_size': "
 "8, 'seated': 1}, {'party': 1, 'timeslot': 1, 'party_size': 8, 'seated': 1}, "
 "{'party': 2, 'timeslot': 1, 'party_size': 8, 'seated': 1}, {'party': 3, "
 "'timeslot': 1, 'party_size': 8, 'seated': 1}, {'party': 4, 'timeslot': 1, "
 "'party_size': 5, 'seated': 1}, {'party': 5, 'timeslot': 1, 'party_size': 6, "
 "'seated': 0}, {'party': 6, 'timeslot': 1, 'party_size': 3, 'seated': 1}, "
 "{'party': 7, 'timeslot': 1, 'party_size': 3, 'seated': 0}, {'

In [11]:
# Helper function

def calc_delta_space(removed_size, added_size):
    removed_tables = table_mapping[removed_size]
    added_tables = table_mapping[added_size]
    
    return 3 * (added_tables - removed_tables)

def calc_delta_space2(added_size):
    added_tables = table_mapping[added_size]
    
    return 3 * added_tables


### Step 7: Run Simulated Annealing

In [12]:
import math
import time

start_time = time.time()

# 1. randomly select which time slot to alter
# 2. randomly select tables to add/remove within the time slot

# TODO pass all these thru the json file
og_revenue = incumbent[8]
T = 0.2*og_revenue
alpha = 0.5
m = 2
# num. iterations
k = 6
iterations = m*k
incumbent_history = []
incumbent.append(0)
incumbent_history.append(incumbent)

print(f"{I}")

with open('log' + RG_HEX_ID + '.txt', 'w') as log_file:
    log_file.write(f"Initialized parameters: T_0 = {T:.2f}, alpha = {alpha}, m = {m}, k = {k}, iterations = {m*k}\n\n")
    
    for iteration in range(iterations):
        which_timeslot = random.randint(0,6)
        #incumbent[which_timeslot] = sorted(incumbent[which_timeslot], key=lambda k: k['seated'], reverse=True)
        print(f"old incumbent? {incumbent[which_timeslot]}")
        print(f"TIMESLOT {which_timeslot}")
        log_file.write(f"RANDOMLY GENERATED TIMESLOT: {which_timeslot}\n")
        # pprint.pprint(f"INCUMBENT FOR TIMESLOT {incumbent[which_timeslot]}")
        
        challenger = incumbent.copy()
        
        first_zero = -1
        
        for index, entry in enumerate(challenger[which_timeslot]):
            if entry['seated'] == 0 and index != 0:
                first_zero = index
                break
        if first_zero == -1:
            failures += 1
            # do nothing because all parties are seated
            print("do nothing")
            # while loop could get us into an infinite loop. 
            # This is the easiest way to mitigate since we want a do-while loop which doesn't exist in Python
            if failures == 10:
                break
            continue
        
        challenger = incumbent.copy()

        # TODO - set this to 0 and added tables to 1 IIF we accept the candidate soln
        print(f"first zero: {first_zero}")
        table_to_remove = random.randint(0, first_zero-1)
        removed_party_size = challenger[which_timeslot][table_to_remove]['party_size']
        removed_table_size = table_mapping[challenger[which_timeslot][table_to_remove]['party_size']]
        removed_space = removed_table_size*3 + 6
        
        # TODO - remember to change this back if needed
        challenger[which_timeslot][table_to_remove]['seated'] = 10
        
        print(f"evaluating removing table #{table_to_remove}")
        log_file.write(f"evaluating removing table #{table_to_remove}\n")
        
        candidate_change_in_space = removed_space * -1
        candidate_change_in_revenue = removed_party_size * marginal_revenue[removed_party_size] * -1
        # Once we remove a table, keep adding new ones until we don't have any room
        # while loop condition - while we can add at least the smallest table to the restaurant. 
        # Smallest party is composed of 2 people, hence table_mapping[2], where party of 2 is mapped to # tables needed
        num_runs = 0
        
        # no matter what, we can only add 2 additional tables
        while(challenger[9][which_timeslot] + table_mapping[2]*3 + 6 + candidate_change_in_space < MAX_SIZE):
            num_runs += 1
            failures = 0

            first_zero = -1
            for index, entry in enumerate(challenger[which_timeslot]):
                if index != 0:
                    if entry['seated'] == 0:
                        first_zero = index
                        break
            if first_zero == -1:
                failures += 1
                # do nothing because all parties are seated
                print("do nothing")
                # while loop could get us into an infinite loop. 
                # This is the easiest way to mitigate since we want a do-while loop which doesn't exist in Python
                if failures == 10:
                    break
                continue

            print(f"FIRST ZERO {first_zero}")
            
            # this is just the index
            table_to_add = random.randint(first_zero, len(challenger[which_timeslot])-1)
            print(f"evaluating adding table #{table_to_add}")
            log_file.write(f"evaluating adding table #{table_to_add}")
            
            added_party_size = challenger[which_timeslot][table_to_add]['party_size']
            candidate_change_in_space += table_mapping[added_party_size]*3 + 6

            print(f"changing 0 to 1")
            challenger[which_timeslot][table_to_add]['seated'] = 1

            # TODO - update challenger's revenue field at the end if accepted
            candidate_change_in_revenue += added_party_size * marginal_revenue[added_party_size]
            
#             # Re-sort affected timeslot in descending order 
#             print(f"sorting...\n before\n{challenger[which_timeslot]}\nafter\n{challenger[which_timeslot]}\n\n")
#             challenger[which_timeslot] = sorted(challenger[which_timeslot], key=lambda k: k['seated'], reverse=True)
            
        log_file.write(f"change in space = {candidate_change_in_space}")
        log_file.write(f"change in revenue = {candidate_change_in_revenue}")

        RNG = random.uniform(0, 1)
        
        print(f"\n\n  current = {challenger[which_timeslot]}\n\n challenger = {challenger[which_timeslot]} \n\n")

        if candidate_change_in_revenue > 0:
            print("ACCEPTING CANDIDATE SOLUTION, OBJ VAL INCREASED")
            log_file.write("ACCEPTING CANDIDATE SOLUTION, OBJ VAL INCREASED\n")
            print(f"delta rev {candidate_change_in_revenue}")

            print(f"changing {challenger[which_timeslot][0]['seated']} to 0")
            challenger[which_timeslot][0]['seated'] = 0
            temp = sorted(challenger[which_timeslot], key=lambda k: k['seated'], reverse=True)
            challenger[which_timeslot] = temp
            
            # accept challenger solution - overwrite incumbent
            challenger[8] = revenue + candidate_change_in_revenue
            challenger[9][which_timeslot] = challenger[9][which_timeslot] + candidate_change_in_space
            challenger[-1] = iteration+1
            
            for i in challenger[which_timeslot]:
                print(f"i in challenger[which_timselot] = {i}")
                if i['seated'] == 10:
                    print("SEATED == 10")
                    i['seated'] = 0
                    print(f"i['seated'] = {i['seated']}")

            incumbent_history.append(challenger)
            print(f"new incumbent? {challenger[which_timeslot]}")
            
        elif RNG < math.exp(candidate_change_in_revenue / T):
            print(f"change in revenue = {candidate_change_in_revenue}, T = {T}")
            print(f"ACCEPTING CANDIDATE WITH PROBABILITY {RNG} < {math.exp(candidate_change_in_revenue / T)}")
            log_file.write(f"ACCEPTING CANDIDATE WITH PROBABILITY {RNG} < {math.exp(candidate_change_in_revenue / T)}\n")

            print(f"changing {challenger[which_timeslot][0]['seated']} to 0")
            challenger[which_timeslot][0]['seated'] = 0
            temp = sorted(challenger[which_timeslot], key=lambda k: k['seated'], reverse=True)
            challenger[which_timeslot] = temp
            
            # accept challenger solution - add incumbent
            challenger[8] = revenue + candidate_change_in_revenue
            print(f"current spaced occupied for timeslot {which_timeslot} = {challenger[9][which_timeslot]}")
            challenger[9][which_timeslot] = challenger[9][which_timeslot] + candidate_change_in_space
            print(f"new spaced occupied for timeslot {which_timeslot} = {challenger[9][which_timeslot]}")
            challenger = challenger.copy()
            challenger[-1] = iteration+1
            
            for i in challenger[which_timeslot]:
                print(f"i in challenger[which_timselot] = {i}")
                if i['seated'] == 10:
                    i['seated'] = 0

            incumbent_history.append(challenger)
            print(f"new incumbent? {incumbent[which_timeslot]}")
        else:
            print("REJECTING CANDIDATE SOLUTION")
            log_file.write("REJECTING CANDIDATE SOLUTION\n")
        print("\n\n")
        


        if (iteration % m == 0):
            T = T*alpha
        print(f"ITERATION = {iteration}")
        print(f"TEMPERATURE = {T}\n\n")
        log_file.write(f"Completed Iteration = {iteration}\n")
        log_file.write(f"Updating T... T_{i} = {T:.2f}\n\n\n")
    
    print("\n\n--- %s seconds ---\n\n" % (time.time() - start_time))
    log_file.write("\n\n--- %s seconds ---\n\n" % (time.time() - start_time))

[8, 10, 13, 15, 16, 15, 10]
old incumbent? [{'party': 0, 'timeslot': 3, 'party_size': 8, 'seated': 1}, {'party': 1, 'timeslot': 3, 'party_size': 8, 'seated': 1}, {'party': 2, 'timeslot': 3, 'party_size': 7, 'seated': 1}, {'party': 3, 'timeslot': 3, 'party_size': 7, 'seated': 1}, {'party': 4, 'timeslot': 3, 'party_size': 7, 'seated': 1}, {'party': 5, 'timeslot': 3, 'party_size': 5, 'seated': 0}, {'party': 6, 'timeslot': 3, 'party_size': 6, 'seated': 0}, {'party': 7, 'timeslot': 3, 'party_size': 3, 'seated': 0}, {'party': 8, 'timeslot': 3, 'party_size': 4, 'seated': 0}, {'party': 9, 'timeslot': 3, 'party_size': 3, 'seated': 0}, {'party': 10, 'timeslot': 3, 'party_size': 2, 'seated': 1}, {'party': 11, 'timeslot': 3, 'party_size': 2, 'seated': 0}, {'party': 12, 'timeslot': 3, 'party_size': 2, 'seated': 0}, {'party': 13, 'timeslot': 3, 'party_size': 2, 'seated': 0}, {'party': 14, 'timeslot': 3, 'party_size': 2, 'seated': 0}]
TIMESLOT 3
first zero: 5
evaluating removing table #1
FIRST ZERO 5


REJECTING CANDIDATE SOLUTION



ITERATION = 6
TEMPERATURE = 12.7375


old incumbent? [{'party': 0, 'timeslot': 2, 'party_size': 7, 'seated': 1}, {'party': 1, 'timeslot': 2, 'party_size': 8, 'seated': 1}, {'party': 2, 'timeslot': 2, 'party_size': 5, 'seated': 1}, {'party': 3, 'timeslot': 2, 'party_size': 6, 'seated': 1}, {'party': 4, 'timeslot': 2, 'party_size': 6, 'seated': 1}, {'party': 5, 'timeslot': 2, 'party_size': 5, 'seated': 1}, {'party': 6, 'timeslot': 2, 'party_size': 6, 'seated': 0}, {'party': 7, 'timeslot': 2, 'party_size': 6, 'seated': 0}, {'party': 8, 'timeslot': 2, 'party_size': 4, 'seated': 0}, {'party': 9, 'timeslot': 2, 'party_size': 3, 'seated': 0}, {'party': 10, 'timeslot': 2, 'party_size': 2, 'seated': 0}, {'party': 11, 'timeslot': 2, 'party_size': 2, 'seated': 0}, {'party': 12, 'timeslot': 2, 'party_size': 2, 'seated': 0}]
TIMESLOT 2
first zero: 6
evaluating removing table #1
FIRST ZERO 6
evaluating adding table #10
changing 0 to 1
FIRST ZERO 6
evaluating adding t

In [13]:
pprint.pprint(incumbent_history)

[[[{'party': 0, 'party_size': 8, 'seated': 1, 'timeslot': 0},
   {'party': 1, 'party_size': 6, 'seated': 1, 'timeslot': 0},
   {'party': 2, 'party_size': 5, 'seated': 10, 'timeslot': 0},
   {'party': 3, 'party_size': 5, 'seated': 1, 'timeslot': 0},
   {'party': 4, 'party_size': 6, 'seated': 1, 'timeslot': 0},
   {'party': 5, 'party_size': 3, 'seated': 1, 'timeslot': 0},
   {'party': 6, 'party_size': 2, 'seated': 1, 'timeslot': 0},
   {'party': 7, 'party_size': 2, 'seated': 1, 'timeslot': 0}],
  [{'party': 0, 'party_size': 8, 'seated': 10, 'timeslot': 1},
   {'party': 1, 'party_size': 8, 'seated': 10, 'timeslot': 1},
   {'party': 2, 'party_size': 8, 'seated': 1, 'timeslot': 1},
   {'party': 3, 'party_size': 8, 'seated': 1, 'timeslot': 1},
   {'party': 4, 'party_size': 5, 'seated': 10, 'timeslot': 1},
   {'party': 5, 'party_size': 6, 'seated': 0, 'timeslot': 1},
   {'party': 6, 'party_size': 3, 'seated': 1, 'timeslot': 1},
   {'party': 7, 'party_size': 3, 'seated': 1, 'timeslot': 1},
   

In [14]:
max_OV = 0
idx = 0
for index, incumbent in enumerate(incumbent_history):
    if incumbent[8] > max_OV:
        print(f"{incumbent[8]}, max = {max_OV}")
        max_OV = incumbent[8]
        idx = index

print(f"index of max incumbent {max_OV}")
#pprint.pprint(f"{incumbent_history[idx]}")
    

1019, max = 0
1036, max = 1019
index of max incumbent 1036


In [15]:
def write_incumbent_to_file(file_name, index, incumbent):
    with open(file_name, 'a') as csv_file:
        writer = csv.writer(csv_file)
        header = []
        for key in incumbent[0][0].keys():
            header.append(key)
        writer.writerow([f"Incumbent {index}"])
        writer.writerow(header)
        for i in range(7):
            writer.writerow([f"Timeslot {i+1}"])
            for j in incumbent[i]:
                li = list(j.values())
                writer.writerow(li)
                #for val in j.values():
                #writer.writerow(j)
        writer.writerow([f"Objective Function value: {incumbent[8]}"])
        for i in range(3):
            writer.writerow('')

file_name = test = "simulated_annealing_" + RG_HEX_ID + ".csv"
for idx, inc in enumerate(incumbent_history):
    write_incumbent_to_file(file_name, idx, inc)
#with open(file_name, 'a') as csv_file:  
 #   writer = csv.writer(csv_file)
    #writer.writerow(["party i", "size of Party i", "timeslot k", "seated (binary)", "added revenue", "cum. revenue"])

In [16]:
for each iteration:
    1. choose a timeslot
    2. pick a candidate table to remove
    3. add tables until we can't anymore
        3.1 find the first party that is not yet seated
        3.2 add that to challenger
        3.3 calc new revenue & space
    4. if revenue inc. or RNG < acceptance prob., then accept candidate solution and append incumbent
        else, don't append incumbent, let it die
    5. make sure challenger removes table to be removed and adds table to added
    6. update iteration no. and temperature (if iteration % 2 == 0)
    
    
    

SyntaxError: invalid syntax (<ipython-input-16-4a4dc008afec>, line 1)

In [ ]:
print(incumbent[which_timeslot])

In [ ]:
inc = incumbent[which_timeslot]
inc.sort()
#inc = sorted(inc, key = lambda k: (k['seated']), reverse=True)
#sorted(lis, key = lambda i: i['age'])
print(inc)